In [158]:
import pandas as pd
import numpy as np

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
features_df = pd.read_csv("features.csv")
stores_df = pd.read_csv("stores.csv")

def load_all():
    train_df = pd.read_csv("train.csv")
    features_df = pd.read_csv("features.csv")
    stores_df = pd.read_csv("stores.csv")
    
    return pd.merge(pd.merge(train_df, features_df), stores_df)

In [314]:
# configuration
SEQ_LEN = 10 # give 10 weeks, predict the sale of next week
FEATURES = ['DayOfYear', 'Weekly_Sales']

In [317]:
# do a simple model to predict price from a store
from datetime import datetime

def convert_day_of_year(x):
    return datetime.strptime(x, "%Y-%m-%d").timetuple().tm_yday

def sliding(x, seq_len):
    t = np.zeros([x.shape[0] - seq_len + 1, seq_len, x.shape[1]])
    for i in range(t.shape[0]):
        t[i,:,:] = x[i:i+seq_len,:]
    
    return t

store_1_sales = train_df[(train_df.Store == 1) & (train_df.Dept == 1)]
store_1_features = features_df[features_df.Store == 1]
store_1 = pd.merge(store_1_sales[["Store", "Dept", "Date", "Weekly_Sales"]], store_1_features)
store_1['DayOfYear'] = store_1['Date'].apply(convert_day_of_year)

# for now only use Weekly_Sales for prediction
store_1 = store_1[FEATURES]

# sliding
result = sliding(store_1.values, SEQ_LEN)

# also split into train & test
num_train = int(len(result) * 0.8)
store_1_train = result[:num_train]
store_1_test = result[-num_train:]

# some normalization
train_mean = store_1_train[0].mean()
train_std = store_1_train[0].std()

store_1_train[0] /= train_std
store_1_test[0] -= train_std

np.random.shuffle(store_1_train)
X_train = store_1_train[:, :SEQ_LEN - 1]
Y_train = store_1_train[:, SEQ_LEN - 1, len(FEATURES) - 1]

X_test = store_1_test[:, :SEQ_LEN - 1]
Y_test = store_1_test[:, SEQ_LEN - 1, len(FEATURES) - 1]

print X_train.shape, Y_train.shape

(107, 9, 2) (107,)


In [337]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Masking
from keras.layers.recurrent import LSTM
import math

def build_model(seq_len, num_features):
    model = Sequential()

    #model.add(Masking(0, input_shape=(seq_len-1, num_features)))
    model.add(LSTM(256, return_sequences=False, input_shape=(seq_len-1, num_features)))
    #model.add(Dropout(0.2))
    #model.add(LSTM(20, return_sequences=False))
    #model.add(Dropout(0.2))
    model.add(Dense(output_dim=math.max(1, num_features-1)))
    model.add(Activation('linear'))
    
    model.compile(loss="mse", optimizer="rmsprop")
    return model

In [338]:
model = build_model(SEQ_LEN, len(FEATURES))

AttributeError: 'module' object has no attribute 'max'

In [327]:
def train(model, X_train, Y_train, X_test):
    epochs = 1000
    predicted = None
    
    try:
        model.fit(
            X_train, Y_train,
            batch_size=512, nb_epoch=epochs, validation_split=0.1,
            verbose=0
        )
        predicted = model.predict(X_test)
    except Exception as ex:
        print ex
    
    return model, predicted

In [ ]:
model, predicted = train(model, X_train, Y_train, X_test)

In [334]:
predicted - Y_test

array([[-23808.13436798, -16489.45436798, -18861.08436798, ...,
        -22698.35436798, -24119.61436798, -27325.15436798],
       [-23808.13436798, -16489.45436798, -18861.08436798, ...,
        -22698.35436798, -24119.61436798, -27325.15436798],
       [-23808.13436798, -16489.45436798, -18861.08436798, ...,
        -22698.35436798, -24119.61436798, -27325.15436798],
       ..., 
       [-23808.13435272, -16489.45435272, -18861.08435272, ...,
        -22698.35435272, -24119.61435272, -27325.15435272],
       [-23808.13435272, -16489.45435272, -18861.08435272, ...,
        -22698.35435272, -24119.61435272, -27325.15435272],
       [-23808.13435272, -16489.45435272, -18861.08435272, ...,
        -22698.35435272, -24119.61435272, -27325.15435272]])